<a href="https://colab.research.google.com/github/BambangSinaga/ai_lab/blob/main/TugasKelompokKe_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mengunduh dataset, load, dan explorasi awal
import pandas as pd
import matplotlib.pyplot as plt

orders = pd.read_csv('kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')
orders.head()

reviews = pd.read_csv('kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv')
reviews.head()

items = pd.read_csv('kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
items.head()


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


# 3. PERSIAPAN DATA
# Gabung order -> review -> item

In [ ]:
df = orders.merge(reviews, on='order_id', how='inner')\
           .merge(items, on='order_id', how='inner')

# Menghapus baris dengan nilai yang hilang dari DataFrame gabungan
df.dropna(inplace=True)

# Memeriksa jumlah nilai yang hilang di setiap kolom
print("Jumlah nilai yang hilang di DataFrame gabungan:")
print(df.isnull().sum())

Jumlah nilai yang hilang di DataFrame gabungan:
order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
review_id                        0
review_score                     0
review_comment_title             0
review_comment_message           0
review_creation_date             0
review_answer_timestamp          0
order_item_id                    0
product_id                       0
seller_id                        0
shipping_limit_date              0
price                            0
freight_value                    0
dtype: int64


# Hitung metrik per produk

In [ ]:
# Buat DataFrame final dengan kolom product_id, jumlah_review, rating_rata2, volume_penjualan
agg_df = df.groupby('product_id').agg({
    'review_score': ['count', 'mean'],
    'order_id': 'count'
}).reset_index()

agg_df.columns = ['product_id', 'jumlah_review', 'rating_rata2', 'volume_penjualan']
agg_df.dropna(inplace=True)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

X = agg_df[['jumlah_review', 'rating_rata2']]
y = agg_df['volume_penjualan']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Koefisien regresi:", model.coef_)
print("Intercept:", model.intercept_)
print("R²:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred))

Koefisien regresi: [1.0000000e+00 3.6237014e-17]
Intercept: 2.4424906541753444e-15
R²: 1.0
RMSE: 1.4910821674995882e-29


In [ ]:
# 5. VISUALISASI INTERAKTIF
import plotly.express as px
fig = px.scatter_3d(agg_df,
                    x='jumlah_review',
                    y='rating_rata2',
                    z='volume_penjualan',
                    color='volume_penjualan',
                    size='jumlah_review',
                    title='Prediksi Volume Penjualan Berdasarkan Review & Rating')
fig.show()